
import numpy as np
import cv2
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import scipy.fftpack 

trdata = 71999
vltdata = 21600
batch = 16 
arr_result = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

training_data = 'nist_final/training'
validation_data = 'nist_final/validation'

model=Sequential()
model.add(Conv2D(32,(3,3),input_shape=(64,64,1),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=36,activation='sigmoid'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

train_datagen=ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                    horizontal_flip = False)

test_datagen=ImageDataGenerator(rescale = 1./255)

training_set=train_datagen.flow_from_directory(directory = training_data,
                                                 target_size = (64, 64),
                                                 color_mode='grayscale',
                                                 batch_size = batch,
                                                 class_mode = 'sparse')

test_set=test_datagen.flow_from_directory(directory = validation_data,
                                            target_size = (64, 64),
                                            color_mode='grayscale',
                                            batch_size = batch,
                                            class_mode = 'sparse')
 
model.fit_generator(training_set,steps_per_epoch = 4500,         
                         epochs = 15,
                         validation_data = test_set,
                         validation_steps = 1350)                 

model.save('fmodelwts.h5')

In [1]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from PIL import Image

In [2]:
arr_out = []
arr_result = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']

In [3]:
model=load_model('fmodelwts.h5')

In [4]:
def sortcnts(cnts):  # to sort the contours left to right
    boundingBoxes = [cv2.boundingREct(c) for c in cnts]
    (cnts,boundingBoxes) = zip(*sorted(zip(cnts,boundingBoxes),key = lambda b:b[1][0], reverse=False))
    return cnts

In [5]:

def test(a,b,c,d,imd):                # to predict the character present in the region of interest
    test=imd[b:b+d,a:a+c]
    _,test_image = cv2.threshold(test,100,255,cv2.THRESH_BINARY)
    test_image= cv2.copyMakeBorder(test_image,10,10,10,10,cv2.BORDER_CONSTANT,value=(255,255,255))
    test_image = cv2.medianBlur(test_image.copy(),3)
    test_image = cv2.resize(test_image.copy(),(64,64),interpolation = cv2.INTER_AREA)
    t = test_image.copy()
    cv2.resize(test_image,(64,64))
    test_image=(image.img_to_array(test_image))/255
    test_image=np.expand_dims(test_image, axis = 0)
    result=model.predict(test_image)  
    np.reshape(result, 36)
    high = np.amax(test_image)
    low = np.amin(test_image)
    if high != low:
        maxval = np.amax(result)
        index = np.where(result == maxval)
        arr_out.append(arr_result[index[1][0]])


In [21]:
def predict(input_img):    
    im = input_img.copy()
    img = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    
    # Code for enhancing the image--------------------------------------------------
    
    blur = cv2.bilateralFilter(img.copy(),9,75,75)
    _, thresh = cv2.threshold(blur.copy(), 100, 255, cv2.THRESH_BINARY) 
    a, contours, h = cv2.findContours(thresh.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    sum = 0
    maxar = 0
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        sum += (w*h)

    
    avg = sum/len(contours)
    maxar = 10000 
    minar = 1000
    for cnt in contours:
        x,y,w,h = cv2.boundingRect(cnt)
        if w*h < maxar and w*h > minar:
            test(x,y,w,h,img)

    final = ""
    i = 0
    for ch in reversed(arr_out):
        i += 1
        final = final+ch

    print('\n',final)

#cv2.waitKey()
#cv2.destroyAllWindows()

In [22]:
img = cv2.imread('test4.jpg')
import matplotlib.pyplot as plt
predict(img)


 AHEAAHEAAHEASESF1111Q8SMY02ZR0
